# Web Complexity

## Importing Files

In [100]:
import json
import matplotlib.pyplot as plt

In [101]:
with open("./output.json") as file:
    websites = json.load(file)
website_len=len(websites)
print("{} websites scanned".format(website_len))

242 websites scanned


## A Glimpse of Generated Data

In [102]:
# Between Classified/Unclassified and Available/Unavailable
info = {
    "Unclassified but Available": 0,
    "Classified but Unavailable (Not 200)": 0,
    "Classified and Available": 0,
    "Unclassified and Unavailable (Not 200)": 0,
    "Repeated Websites":0,
    "Average Rank":0
}
rank_arr=[]
webdetails={}
webcategory={}
mime_used={}
for site in websites:
    try:
        webdetails[site["name"]]+=1
    except:
        webdetails[site["name"]]=1
    try:
        webcategory[site["category"]]+=1
    except:
        webcategory[site["category"]]=1
    if(site["category"]=="Unclassified"):
        if(site["reqcode"]=="N/A"):
            info["Unclassified and Unavailable (Not 200)"]+=1
        else:
            info["Unclassified but Available"]+=1
    else:
        if(site["reqcode"]=="N/A"):
            info["Classified but Unavailable (Not 200)"]+=1
        else:
            info["Classified and Available"]+=1
    if(webdetails[site["name"]]>1):
        info["Repeated Websites"]+=1
    info["Average Rank"]+=site["rank"]
    rank_arr.append((site["rank"],site["name"]))
    # for request in site["reqdetails"]:
info["Average Rank"]//=count
for t in info.keys():
    print("{} : {}".format(t,info[t]))

Unclassified but Available : 43
Classified but Unavailable (Not 200) : 0
Classified and Available : 102
Unclassified and Unavailable (Not 200) : 97
Repeated Websites : 0
Average Rank : 243358


In [103]:
print("\nCategory-wise splitup:")
for t in webcategory.keys():
    print("{}: {}".format(t,webcategory[t]))


Category-wise splitup:
Unclassified: 140
Entertainment: 7
Business/Economy: 23
Technology/Internet: 13
Health: 8
Suspicious: 4
News: 2
Education: 2
Sports/Recreation: 2
Shopping: 2
Society/Daily Living: 1
Audio/Video Clips: 2
Government/Legal: 4
Reference: 5
Real Estate: 1
Charitable/Non-Profit: 2
Gambling: 1
Newsgroups/Forums: 2
Pornography: 1
Finance: 1
Office/Business Applications: 1
Travel: 6
Restaurants/Food: 3
Software Downloads: 2
TV/Video Streams: 1
Social Networking: 2
Media Sharing: 2
Games: 1
Brokerage/Trading: 1


In [104]:
rank_arr.sort(key=lambda x:x[0])
print("{}: Max Rank".format(rank_arr[len(rank_arr)-1]))

(998531, 'xqcisr.buzz'): Max Rank


## Separating Websites into Buckets
Because we didnt have the complete means to collect the data, we've not been able to 

In [105]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def buckets_generate(arr,count):
    old_chunk=[]
    for i in range(0,website_len,10):
        chunk=list(chunks(arr,website_len-i))
        if(len(old_chunk)>=count and len(chunk)!=len(old_chunk)):
            return old_chunk
        old_chunk=chunk

In [106]:
buckets=buckets_generate(rank_arr,4)
for index,bucket in enumerate(buckets):
    print("Bucket {}: {} websites from rank {} to {}".format(index,len(bucket),min(bucket[0][0],bucket[len(bucket)-1][0]),max(bucket[0][0],bucket[len(bucket)-1][0])))

Bucket 0: 62 websites from rank 2 to 75553
Bucket 1: 62 websites from rank 76201 to 161459
Bucket 2: 62 websites from rank 161847 to 299200
Bucket 3: 56 websites from rank 312449 to 998531


## Content Complexity
### Number of Object Requests Made

In [ ]:
def num_of_req(bucket):
    for rank,site in bucket:
        